The update will take place in 7 steps which are described below.

In [4]:
import pandas as pd
import numpy as np
import pymrio
import gzip
import pickle
import re

#### 1. ecospold2matrix

First step, make sure you have extracted the new version of ecoinvent with ecospold2matrix as pandas dataframes.

Then load the pickle into the notebook

In [2]:
with gzip.open('C://Users/Maxime/Desktop/Thesis/Bdd/Ecoinvent/python/ecoinvent3.7.1.cutoff/ecoinvent3.7.1.cut-offPandas_symmNorm.gz.pickle','rb') as f:
    ecoinvent = pd.read_pickle(f)

#### 2. pymrio

Same thing with the new version of exiobase. Extract it with pymrio.

Load exiobase in the notebook too

In [5]:
io = pymrio.parse_exiobase3('C://Users/Maxime/Desktop/Thesis/Bdd/EXIOBASE/monetary/exiobase3.8.1/IOT_2019_pxp/')

#### 3. Folder for the new version

Create a folder within src/Data/ for the new version, e.g., for ecoinvent3.7.1 and exiobase3.8.1, create a folder eco3.7.1_exio3. No need to specify exio3.8.1 as exiobase does not change format within a same version number (and we are thankful for that!)

#### 4. Filter.xlsx

Next we create the Filter.xlsx file for the new version.

##### 4.1 Hybridized processes and others

We define the processes to be hybridized and most of the ones not to be hybridized, and the reason why they are not hybridized. The piece of code below takes care of that for you.

In [6]:
list_to_hyb = []
list_not_to_hyb = []
dummyprocesses = []
null_price = []
listmarket = []
listnottransacted = []
listguillotine = []


listproduct = ['sugar beet pulp', 'shavings', 'calcium chloride', 'pulverised lignite', 'heat pump, 30kW',
               'planning', 'saw dust', 'limestone, crushed', 'water, ultrapure', 'components for',
               'burned in refinery furnace', 'stone meal', 'grass', 'miscanthus', 'compressed',
               'evaporation', 'for planting', 'from zinc', 'zinc concentrate',
               'completely softened', 'tap water', 'heavy water', 'water, deionised', 'water, decarbonised',
               'ground granulated', 'deep well', 'collection service']
listactivity = ['basic slag', 'start-up', 'metals recovery', 'venting', 'nutrient supply',
                'recultivation', 'extrusion', 'excavation', 'transformation',
                'water production','thermoforming', 'blow moulding', 'sorting',
                'natural gas pressure reduction from high to low pressure']
listaddmarket = ['import', 'to generic market', 'to niche market', 'production mix', 'aluminium industry',
                 'cobalt industry','to market','electricity production, high voltage, for Swiss Federal Railways']

# calculating the sum of cost, we will compare it to the price and assert it's lower than the price
A_ff = ecoinvent['A'].copy()
A_ff = A_ff.fillna(0)
matrix = pd.concat([ecoinvent['PRO'].price.fillna(0)]*len(ecoinvent['PRO']),axis=1)
matrix.columns = ecoinvent['PRO'].index.tolist()
A_ff = A_ff.multiply(matrix)

for i in range(0, len(ecoinvent['PRO'])):
    try:
        # first, identify dummy processes, processes that do not have any inputs
        if ecoinvent['A'].loc[:,ecoinvent['PRO'].index[i]].sum() == 0:
            if ecoinvent['PRO'].index[i] not in list_not_to_hyb:
                list_not_to_hyb.append(ecoinvent['PRO'].index[i])
                dummyprocesses.append(ecoinvent['PRO'].index[i])
        # then those that do not have any price (we obviously cannot hybridize them)
        if ecoinvent['PRO'].price[i] == 0:
            if ecoinvent['PRO'].index[i] not in list_not_to_hyb:
                list_not_to_hyb.append(ecoinvent['PRO'].index[i])
                null_price.append(ecoinvent['PRO'].index[i])
        # market processes identification
        if (ecoinvent['PRO'].activityType[i] == 1
                or ecoinvent['PRO'].activityType[i] == 4
                or ecoinvent['PRO'].activityType[i] == 10):
            if ecoinvent['PRO'].index[i] not in list_not_to_hyb:
                list_not_to_hyb.append(ecoinvent['PRO'].index[i])
                listmarket.append(ecoinvent['PRO'].index[i])
        for j in range(0,len(listaddmarket)):
            if listaddmarket[j] in ecoinvent['PRO'].activityName[i]:
                if ecoinvent['PRO'].index[i] not in list_not_to_hyb:
                    list_not_to_hyb.append(ecoinvent['PRO'].index[i])
                    listmarket.append(ecoinvent['PRO'].index[i])
                    break
        # non transacted goods/services identification
        for j in range(0, len(listproduct)):
            if listproduct[j] in ecoinvent['PRO'].productName[i]:
                if ecoinvent['PRO'].index[i] not in list_not_to_hyb:
                    list_not_to_hyb.append(ecoinvent['PRO'].index[i])
                    listnottransacted.append(ecoinvent['PRO'].index[i])
                    break
        for k in range(0, len(listactivity)):
            if listactivity[k] in ecoinvent['PRO'].activityName[i]:
                if ecoinvent['PRO'].index[i] not in list_not_to_hyb:
                    list_not_to_hyb.append(ecoinvent['PRO'].index[i])
                    listnottransacted.append(ecoinvent['PRO'].index[i])
                    break
        # identify intra-sectorial processes
        if (len(re.findall(r"[a-z]*[i][n][g]\b", ecoinvent['PRO'].activityName[i], flags=0)) != 0
            and 'production' not in ecoinvent['PRO'].activityName[i]
            and 'treatment' not in ecoinvent['PRO'].activityName[i]
            and 'waste' not in ecoinvent['PRO'].activityName[i]
            and 'construction' not in ecoinvent['PRO'].activityName[i]
            and 'installation' not in ecoinvent['PRO'].activityName[i]
            and 'plant' not in ecoinvent['PRO'].activityName[i]
            and 'facility' not in ecoinvent['PRO'].activityName[i]
            and ', building' not in ecoinvent['PRO'].activityName[i]
            and 'transport' not in ecoinvent['PRO'].activityName[i]
            and 'capture' not in ecoinvent['PRO'].activityName[i]
            or 'ventilation' in ecoinvent['PRO'].activityName[i]
            and 'production' not in ecoinvent['PRO'].activityName[i]
            and 'treatment' not in ecoinvent['PRO'].activityName[i]
            or 'energy' in ecoinvent['PRO'].activityName[i]
            and 'production' not in ecoinvent['PRO'].activityName[i]
            or ecoinvent['PRO'].unitName[i] == 'ha'
            or (ecoinvent['PRO'].unitName[i] == 'hour'
                and 'helicopter' not in ecoinvent['PRO'].productName[i])
            or len(re.findall(r'\b[s][e][e][d]', ecoinvent['PRO'].productName[i])) != 0
            or len(re.findall(r"^[0][1][6]", ecoinvent['PRO'].ISIC[i])) != 0
            or ('operation' in ecoinvent['PRO'].activityName[i]
                and ('mine' not in ecoinvent['PRO'].activityName[i]
                     and 'petroleum' not in ecoinvent['PRO'].activityName[i]
                     and 'quarry' not in ecoinvent['PRO'].activityName[i]
                     and 'pit' not in ecoinvent['PRO'].activityName[i]))
            or 'maintenance' in ecoinvent['PRO'].productName[i]
                and ('bus' not in ecoinvent['PRO'].productName[i]
                     and 'vehicle' not in ecoinvent['PRO'].productName[i]
                     and 'lorry' not in ecoinvent['PRO'].productName[i]
                     and 'passenger car' not in ecoinvent['PRO'].productName[i]
                     and 'electric bicycle' not in ecoinvent['PRO'].productName[i]
                     and 'scooter' not in ecoinvent['PRO'].productName[i]
                     and 'trawler' not in ecoinvent['PRO'].productName[i])):
            if ecoinvent['PRO'].index[i] not in list_not_to_hyb:
                list_not_to_hyb.append(ecoinvent['PRO'].index[i])
                listnottransacted.append(ecoinvent['PRO'].index[i])
        # guillotines, quality of process is questionable
        if (ecoinvent['PRO'].productName[i] != 'electricity, high voltage'
            and ecoinvent['PRO'].productName[i] != 'electricity, medium voltage'
            and ecoinvent['PRO'].productName[i] != 'electricity, low voltage'):
            if (ecoinvent['PRO'].price[i] > 100000
                    or A_ff.iloc[:, i].sum() > ecoinvent['PRO'].price[i]
                    or A_ff.iloc[:, i].sum() < 0.01 * ecoinvent['PRO'].price[i]):
                if ecoinvent['PRO'].index[i] not in list_not_to_hyb:
                    list_not_to_hyb.append(ecoinvent['PRO'].index[i])
                    listguillotine.append(ecoinvent['PRO'].index[i])

    except TypeError:
        if ecoinvent['PRO'].index[i] not in list_not_to_hyb:
            print('by default the process: '+ecoinvent['PRO'].index[i]+
                  ' was put in listnottransacted, because no sector has been entered.')
            list_not_to_hyb.append(ecoinvent['PRO'].index[i])
            listnottransacted.append(ecoinvent['PRO'].index[i])

for i in range(0, len(ecoinvent['PRO'])):
    if ecoinvent['PRO'].index[i] not in list_not_to_hyb:
        list_to_hyb.append(ecoinvent['PRO'].index[i])

##### 4.2 Potential new geographies

We identify the potential new geographies of the new version of ecoinvent. Only hybridized processes are scanned, as the others ones don't matter.

In [8]:
dict1 = open('C://Users/Maxime/Desktop/Thesis/Modules_Python/pylcaio/src/Data/eco3.7.1_exio3/geography_replacements.txt')
dict1 = eval(dict1.read())
dict2 = open('C://Users/Maxime/Desktop/Thesis/Modules_Python/pylcaio/src/Data/eco3.7.1_exio3/countries_per_regions.txt')
dict2 = eval(dict2.read())
list3 = open('C://Users/Maxime/Desktop/Thesis/Modules_Python/pylcaio/src/Data/eco3.7.1_exio3/countries.txt')
list3 = eval(list3.read())

covered_geos = list(dict1.keys())+list(dict2.keys())+list3

set(ecoinvent['PRO'].loc[[i for i in ecoinvent['PRO'].index if (
    ecoinvent['PRO'].loc[i,'geography'] not in covered_geos and i in list_to_hyb)]].geography)-set(['RoW'])

set()

If the final set is not empty, those are new geographies that are used by ecoinvent. You then have to include those geographies in the different files used for the hybridization.

- If it's a country that is aggregated in the rest-of-the-world regions of exiobase incorporate it in the "geography_replacements.txt" file
- Similarly, if it's a subregion of an existing country in exiobase, (e.g., CA-QC for Quebec) incorporate it in the "geography_replacements.txt" file as well
- If it's a new region (e.g., Europe without Austria) translate that new region in terms of the countries of exiobase in the "countries_per_regions.txt" file

##### 4.3 Uncovered geographies

By default, pylcaio does not hybridize processes for which the country is not in exiobase. So basically, countries that are regrouped in the different rest-of-the-world regions of exiobase (i.e., WA, WE, WF, WL, WM) are not hybridized.

In [9]:
ecoinvent['PRO']['io_geo'] = [dict1[i] if i in dict1 else i for i in ecoinvent['PRO'].geography]
uncoveredgeoDF = ecoinvent['PRO'].loc[[i for i in ecoinvent['PRO'].index if (ecoinvent['PRO'].loc[i,'io_geo'] in ['WA','WE','WF','WL','WM']
                    and i in list_to_hyb)],['price', 'geography','productName', 'activityName']]
# they are not hybridized anymore so remove them from the list
for process in uncoveredgeoDF.index:
    list_to_hyb.remove(process)

##### 4.4 Write the Excel file itself

Finally we write the Excel file. The file needs to be placed within the previously created folder for the new hybrid-ecoinvent version. **_Do not change the names of the Excel tabs._**

In [ ]:
hybridizedDF = ecoinvent['PRO'].loc[list_to_hyb,['price', 'geography','productName', 'activityName']]
marketDF = ecoinvent['PRO'].loc[listmarket,['price', 'geography','productName', 'activityName']]
nottransactedDF = ecoinvent['PRO'].loc[listnottransacted,['price', 'geography','productName', 'activityName']]
guillotineDF = ecoinvent['PRO'].loc[listguillotine,['price', 'geography','productName', 'activityName']]
dummyDF = ecoinvent['PRO'].loc[dummyprocesses,['price', 'geography','productName', 'activityName']]
nullpriceDF = ecoinvent['PRO'].loc[null_price,['price', 'geography','productName', 'activityName']]

writer = pd.ExcelWriter('C://Users/Maxime/Desktop/Thesis/Modules_Python/pylcaio/src/Data/eco3.7.1_exio3/Filter.xlsx', engine='xlsxwriter')
hybridizedDF.to_excel(writer, sheet_name='Hybridized')
marketDF.to_excel(writer, sheet_name='Market')
nottransactedDF.to_excel(writer, sheet_name='Not commercialized')
guillotineDF.to_excel(writer, sheet_name='Poor quality')
dummyDF.to_excel(writer, sheet_name='Empty processes')
nullpriceDF.to_excel(writer, sheet_name='No price')
uncoveredgeoDF.to_excel(writer, sheet_name='Uncovered geography')
writer.save()

#### 5. Concordances

Here we generate the Product_concordances.xlsx file necessary to match ecoinvent processes to exiobase sectors.

##### 5.1 Update existing file

We first update the metadata of the Product_concordances.xlsx file of the current version (i.e., not the new one)

To do this we need to fetch a file called "change_report_annex" which is available on ecoinvent's website.

In [10]:
current_version = '3.6'
new_version = '3.7.1'
change_report = pd.read_excel('C://Users/Maxime/Desktop/change_report_annex_'+current_version+'_'+new_version+'_public.xlsx','qualitative changes')

Also need to load the previous Product_concordances.xlsx file

In [12]:
codes = pd.read_excel('C://Users/Maxime/Desktop/Thesis/Modules_Python/pylcaio/src/Data/eco'+current_version+'_exio3/Product_Concordances.xlsx','Description_Exiobase')
prod_concordance = pd.read_excel('C://Users/Maxime/Desktop/Thesis/Modules_Python/pylcaio/src/Data/eco'+current_version+'_exio3/Product_Concordances.xlsx','Concordance per product')
act_concordance = pd.read_excel('C://Users/Maxime/Desktop/Thesis/Modules_Python/pylcaio/src/Data/eco'+current_version+'_exio3/Product_Concordances.xlsx','Concordance per activity')

We then update the metadata for the concordance per product tab

In [13]:
# selecting processes for which the reference product changed
try:
    if "FALSE" in change_report.loc[:,'reference product different'].tolist():
        changed_names = change_report.loc[[i for i in change_report.index if change_report.loc[i,'reference product different']=='TRUE']]
    elif False in change_report.loc[:,'reference product different'].tolist():
        changed_names = change_report.loc[[i for i in change_report.index if change_report.loc[i,'reference product different']==True]]
except KeyError:
    print('Column used got renamed.')
    
# extracting the changed data as a dictionary
try:
    names_dict = dict(zip(changed_names.loc[:,'reference product - '+current_version],changed_names.loc[:,'reference product - '+new_version]))
except KeyError:
    print('Column used got renamed.')

# make the changes inside the file
prod_concordance.loc[:,'productName'] = [names_dict[i] if i in names_dict else i for i in prod_concordance.productName]

# get the product ids from ecoinvent
updated_prod_concordance = prod_concordance.loc[:,['productName','Concordance']].merge(ecoinvent['PRO'].loc[:,['productId','productName']], how='left').drop_duplicates()

#reordering
updated_prod_concordance = updated_prod_concordance.reindex(['productId','productName','Concordance'],axis=1)

and for the concordance per activity tab

In [19]:
# selecting processes for which the reference product changed
try:
    if "FALSE" in change_report.loc[:,'activityName different'].tolist():
        changed_names = change_report.loc[[i for i in change_report.index if change_report.loc[i,'activityName different']=='TRUE']]
    elif False in change_report.loc[:,'activityName different'].tolist():
        changed_names = change_report.loc[[i for i in change_report.index if change_report.loc[i,'activityName different']==True]]
except KeyError:
    print('Column used got renamed.')

# extracting the changed data as a dictionary
try:
    names_dict = dict(zip(changed_names.loc[:,'activityName - '+current_version],changed_names.loc[:,'activityName - '+new_version]))
except KeyError:
    print('Column used got renamed.')

# make the changes inside the file
act_concordance.loc[:,'activityName'] = [names_dict[i] if i in names_dict else i for i in act_concordance.activityName]

# get the product ids from ecoinvent
updated_act_concordance = act_concordance.loc[:,['activityName','Concordance']].merge(ecoinvent['PRO'].loc[:,['activityNameId','activityName']], how='left').drop_duplicates()

#reordering
updated_act_concordance = updated_act_concordance.reindex(['activityNameId','activityName','Concordance'],axis=1)

Finally we rewerite the file with the updated metadata.

In [ ]:
writer = pd.ExcelWriter('C://Users/Maxime/Desktop/Thesis/Modules_Python/pylcaio/src/Data/eco'+new_version+'_exio3/Product_concordances.xlsx', 
                        engine='xlsxwriter')

codes.to_excel(writer, sheet_name='Description_Exiobase', index=False)
updated_prod_concordance.to_excel(writer, sheet_name='Concordance per product', index=False)
updated_act_concordance.to_excel(writer, sheet_name='Concordance per activity', index=False)
writer.save()

##### 5.2 New processes

The matching for new processes has to be performed manually. Nevertheless, here is some code that ease the process by identifying what are the new processes for which a matching is needed.

In [20]:
# load the filter we created previously
filterhyb = pd.read_excel('C://Users/Maxime/Desktop/Thesis/Modules_Python/pylcaio/src/Data/eco'+new_version+'_exio3/Filter.xlsx')

# identify the new processes
new_processes = filterhyb.loc[[i for i in filterhyb.index if (filterhyb.loc[i,'productName'] not in updated_prod_concordance.productName.tolist() and
                               filterhyb.loc[i,'activityName'] not in updated_act_concordance.activityName.tolist())]]

# isolate the concordances that need to be dealt with manually
concordances_to_do = ecoinvent['PRO'].loc[new_processes.loc[:,'index'], ['productId','productName']].reset_index().drop(['index'],axis=1)
concordances_to_do = concordances_to_do.drop_duplicates()

Now you can copy and paste them to the updated Product_concordancex.xlsx file and match them manually based on the productName and on the exiobase code (see Description_Exiobase for the codes). Here is a link that can help for the matching: https://ec.europa.eu/eurostat/ramon/nomenclatures/index.cfm?TargetUrl=LST_NOM_DTL_LINEAR&StrNom=NACE_1_1&StrLanguageCode=EN

In [21]:
concordances_to_do

,productId,productName
0,beb1b4ef-e2ad-40e3-934f-10a2c0006eed,"rare earth oxide concentrate, 50% REO"
1,1ec16786-a836-54a3-896a-f41ea3a9f9ec,ytterbium oxide
3,1020c079-f3c3-5699-8fc9-72559914756a,dysprosium oxide
4,43df1b02-7f7b-5f66-9cc6-f93a64e4b0c3,holmium oxide
5,7495e6d0-60ec-5d30-9686-4dba467f8634,terbium oxide
6,770a59c7-501a-56a8-988a-86d8a2d8f309,lutetium oxide
7,90542231-d3f9-53df-ba02-2adbecf59e9c,gadolinium oxide
8,a392d309-fdfb-58be-93e4-8c0b843f548c,thulium oxide
9,e8e6fc47-a074-5f74-ab66-270ebe4e0f25,erbium oxide
17,9668470a-28c8-55ed-a2fd-8a6f1aeb3d68,praseodymium-neodymium oxide


#### 7. Miscellaneous